In [45]:
# alpha:finish consulting; beta:new coming;
# overall timer format: (timepoint, event, stationid, serverid)

class Station:
    def __init__(self, stationid, overall_timer, process_times, n_server=0, n_buffer=0):
        self.stationid = stationid
        self.overall_timer = overall_timer
        self.s = n_server
        self.c = n_buffer
        self.wait = False
        self.status = [0, [0]*n_server] # 1 for occupied
        self.process_times = process_times
        self.next_station = None
    
    def setnext(self, next_station):
        self.next_station = next_station
        
    def update(self, timeslot):
        event= timeslot[1]
        if event == 'alpha':
            serverid = timeslot[3]
            self.alpha(serverid)
        else:
            self.beta()
        del(self.overall_timer[0])
        self.overall_timer.sort(key= lambda k:k[0])
        
    def alpha(self, serverid):
        if self.next_station == None:
            if self.status[0] > 0: # wqiting line is not empty
                self.status[0] -= 1
                try:
                    self.overall_timer.append((self.process_times[0], 'alpha', self.stationid, serverid))
                    del(self.process_times[0])
                except:
                    print("empty process time list for station", self.stationid, "server", serverid)
            else:
                self.status[1][serverid] = 0
                del(self.overall_timer[0])
                return True
        else:
            if self.next_station.beta():
                if self.status[0] > 0: # wqiting line is not empty
                    self.status[0] -= 1
                    try:
                        self.overall_timer.append((self.process_times[0], 'alpha', self.stationid, serverid))
                        del(self.process_times[0])
                    except:
                        print("empty process time list for station", self.stationid, "server", serverid)
                else:
                    self.status[1][serverid] = 0
                    return True
            else:
                del(self.overall_timer[0])
                for i in range(len(self.overall_timer)):
                    if self.overall_timer[i][2]==self.stationid+1:
                        self.overall_timer.append(self.overall_timer[i][0], 'alpha', self.stationid, serverid)
                        return True
                self.overall_timer.append((self.overall_timer[0][0], 'alpha', self.stationid, serverid))
            
    def beta(self):
        if self.status[0] > 0: 
            if self.status[0]<self.c:
                self.status[0]+=1
                return True
            else:
                return False
        else:
            i = 0
            while i < self.s:
                if self.status[1][i]==0:
                    self.status[1][i]=1
                    self.overall_timer.append((self.process_times[0], 'alpha', self.stationid, i))
                    self.process_times.remove(self.process_times[1])
                    break
                else:
                    i += 1
            if i == self.s:
                self.status[0]+=1
            return True
        

In [46]:
import numpy as np
class simulation:
    def run():
        timer = []
        t = np.random.poisson(2.1, 20)
        for i in range(1):
            timer.append((t[i], 'beta', 1))
            
        stations = []
        stations.append(Station(stationid=0, overall_timer=timer, process_times=list(np.random.exponential(2.9, 20)), n_server=1, n_buffer=5))
        stations.append(Station(stationid=1, overall_timer=timer, process_times=list(np.random.exponential(2.2, 20)), n_server=1, n_buffer=6))
        stations.append(Station(stationid=2, overall_timer=timer, process_times=list(np.random.exponential(2.2, 20)), n_server=1, n_buffer=6))
        stations.append(Station(stationid=3, overall_timer=timer, process_times=list(np.random.exponential(2.8, 20)), n_server=1, n_buffer=3))
        stations[2].setnext(stations[3])
        stations[1].setnext(stations[2])
        stations[0].setnext(stations[1])
        
        tt = 0
        while len(timer)>0:
            print(timer)
            tt = timer[0][0]
            stations[timer[0][2]].update(timer[0])
        return tt
            

In [47]:
simulation.run()

[(3, 'beta', 1)]
[(1.0116051270034565, 'alpha', 1, 0)]
[(0.06917904218211333, 'alpha', 2, 0)]
[(5.656826502753527, 'alpha', 3, 0)]


IndexError: list assignment index out of range